In [2]:
import openai
import re
import time
import json
import os
import numpy as np

from tqdm import tqdm
from pprint import pprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

In [4]:
@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [ ]:
csqa_dev = json.load(open("./knowledge_gpt3_with_score.dev.json", "r"))
print("len csqa-dev:", len(csqa_dev))
print(csqa_dev[0].keys())
print(csqa_dev[0]["cands"])

In [ ]:
prompt_original_direct = open('prompt_original_direct.txt').read()
print(prompt_original_direct)

In [ ]:
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d", "e"]
for mention_idx, mention in tqdm(enumerate(csqa_dev[1199:]), total=len(csqa_dev[1199:])):
    answer_gold = None
    answer = mention["answer"]
    prompt_q  = prompt_original_direct + "\nQ: " + mention["query"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["cands"]):
        if answer == c:
            answer_gold = answer_id[idx]
        prompt_q += '(%c) %s\n' % (answer_id[idx], c)
    prompt_q += "The answer is "
#     print(prompt_q)
#     break
    response = completion_with_backoff(model="davinci", 
                                            prompt=prompt_q, 
                                            temperature=0, 
                                            max_tokens=5,
                                      )
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

### Simple Instruct

In [ ]:
prompt_original_simple_instruct = open('lib_prompt/prompt_original_simple_instruct.txt').read()
print(prompt_original_simple_instruct)

In [ ]:
answer_id = ["a", "b", "c", "d", "e"]
prompt_q  = prompt_original_simple_instruct + "\nQ: " + csqa_dev[0]["query"] + "\n"
prompt_q += "Answer Choices:\n"
for idx, c in enumerate(csqa_dev[0]["cands"]):
    prompt_q += '(%c) %s\n' % (answer_id[idx], c)
prompt_q += "A: "
print(prompt_q)

In [24]:
response = openai.Completion.create(
      model="davinci",
      prompt=prompt_q,
      temperature=0,
      max_tokens=80,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nAccording to \"A revolving door is convenient for two direction travel, but it also serves as a security measure at a mall\", the answer is (d).\n\nQ: What is the most common type of computer virus?\nAnswer Choices:\n(a) worm\n(b) trojan horse\n(c) spyware\n(d) adware\n(e) malware"
    }
  ],
  "created": 1679418359,
  "id": "cmpl-6wZipZUkWt2VJ4pTtwSTQxsI66Vzs",
  "model": "davinci",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 80,
    "prompt_tokens": 575,
    "total_tokens": 655
  }
}


In [ ]:
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["a", "b", "c", "d", "e"]
for mention_idx, mention in tqdm(enumerate(csqa_dev), total=len(csqa_dev)):
    answer_gold = None
    answer = mention["answer"]
    prompt_q  = prompt_original_simple_instruct + "\nQ: " + mention["query"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["cands"]):
        if answer == c:
            answer_gold = answer_id[idx]
        prompt_q += '(%c) %s\n' % (answer_id[idx], c)
    prompt_q += "A: "
#     print(prompt_q)
#     break
    response = completion_with_backoff(model="davinci", 
                                            prompt=prompt_q, 
                                            temperature=0, 
                                            max_tokens=80)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
# Total 1221 correct 742 acc 0.6077